In [1]:
import time, os
import argparse, configparser
import numpy as np
from PIL import Image
from numba import jit, prange

In [2]:
from dask.distributed import Client, progress
from dask import delayed
client = Client(n_workers=8, threads_per_worker=2, memory_limit='2GB')
client

distributed.diskutils - INFO - Found stale lock file and directory '/opt/notebooks/dask-worker-space/worker-1ntsp85k', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/notebooks/dask-worker-space/worker-d1t8nctk', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/notebooks/dask-worker-space/worker-xvtltuv_', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/notebooks/dask-worker-space/worker-i921ppta', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/notebooks/dask-worker-space/worker-0ppf_pd1', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/notebooks/dask-worker-space/worker-p65_ekb6', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/notebooks/dask-worker-space/worker-ugr5miji', purging
distributed.diskutils - INFO - Found stale lock file and directory '/opt/notebooks/dask-worker-space/worker-hwm

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41293,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:36425,Total threads: 2
Dashboard: http://127.0.0.1:43003/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:42455,


In [52]:
client.shutdown()

In [3]:
#@delayed
def mandelbrot_2(iters,z,c,image,iters_max):
    while abs(z) <= 2 and iters < iters_max:
        z = z*z + c
        iters += 1

    return iters
#@delayed
def mandelbrot_pixel(real_min, real_max, imag_min, imag_max, iters_max, image,real_step,imag_step):
    for y in prange(0, height) :
        for x in prange(0, width):              

            # a point in a complex plane corrsponding to the pixel (x, y)
            real = real_min + real_step * x
            imag = imag_min + imag_step * y
            c = complex(real, imag)

            # check for convergence
            z = complex(0, 0)
            iters = 0
            iters=mandelbrot_2(iters,z,c,image,iters_max)
             # color pixel in HSV scheme
            image[y, x] = (iters % 256, 255, 255 * (iters < iters_max))
  
            
#@jit(nopython = True, parallel = True, cache = True)
#@delayed
def mandelbrot_colors(real_min, real_max, imag_min, imag_max, iters_max, image):

    # image size in pixels
    width = image.shape[1]
    height = image.shape[0]

    # pixel size in complex plane
    real_step = (real_max - real_min) / width
    imag_step = (imag_max - imag_min) / height
    
    mandelbrot_pixel(real_min, real_max, imag_min, imag_max, iters_max, image,real_step,imag_step)
 #   print(type(image))
    # check convergence of each pixel in the image
    """for y in prange(0, height):
        for x in prange(0, width):              

            # a point in a complex plane corrsponding to the pixel (x, y)
            real = real_min + real_step * x
            imag = imag_min + imag_step * y
            c = complex(real, imag)

            # check for convergence
            z = complex(0, 0)
            iters = 0
            while abs(z) <= 2 and iters < iters_max:
                z = z*z + c
                iters += 1

            # color pixel in HSV scheme
            image[y, x] = (iters % 256, 255, 255 * (iters < iters_max))"""

# end mandelbrot_colors



In [4]:

# mandelbrot creates an image array of the Mandelbrot set
#   real_min, real_max, imag_min, imag_max: bounds of the complex plane
#   iters_max: the maximum number of iterations
#   width, height: size of the final image
list_a=[]
@delayed
def mandelbrot(real_min, real_max, imag_min, imag_max, iters_max, width, height):

    # allocate image array
    image = np.zeros((height, width, 3), dtype = np.uint8)

    # process image
    mandelbrot_colors(real_min, real_max, imag_min, imag_max, iters_max, image)
    # return image array
    return image

In [5]:
# parse arguments

config_file = "default.conf"#args['config']
import dask
# parse config file
config = configparser.ConfigParser()
if os.path.isfile(config_file):
    config.read(config_file)
real_min = config.getfloat('AREA', 'real_min', fallback = -2.5)
real_max = config.getfloat('AREA', 'real_max', fallback = +1.5)
imag_min = config.getfloat('AREA', 'imag_min', fallback = -1.125)
imag_max = config.getfloat('AREA', 'imag_max', fallback = +1.125)
iters_max = config.getint('ITERATIONS', 'max', fallback =256)
width = config.getint('IMAGE', 'width', fallback = 3840)
height = config.getint('IMAGE', 'height', fallback = 2160)
name = config.get('IMAGE', 'name', fallback = 'mandelbrot.jpg')

# main processing
t = time.time()
image=mandelbrot(real_min, real_max, imag_min, imag_max, iters_max, width, height)
image=dask.compute(image)[0]


t = time.time() - t
# save image
Image.fromarray(image, mode='HSV').convert('RGB').save(name)    

# printout
print('PAR: size:', (width, height), 'iterations:', iters_max, 'time:', round(t, 3), "s")    

# end main

PermissionError: [Errno 13] Permission denied: 'mandelbrot.jpg'

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting

In [6]:
"""from distributed import Executor
e=Executor(client.cluster)
e"""

/opt/conda/lib/python3.9/site-packages/distributed/client.py:4785: UserWarning: Executor has been renamed to Client
  warnings.warn("Executor has been renamed to Client")


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45273,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:41301,Total threads: 2
Dashboard: http://127.0.0.1:35465/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:45627,


In [ ]:
e.map()